In [3]:
import jupyter
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import scipy
from sklearn import tree
from sklearn.datasets import load_iris
from classes.data import Data


In [4]:
def load_data(file):
    data = []
    with open(file) as f:
        for line in f:
            data.append(line.strip("\n").split(","))
    return data

def decision_tree(data, classLabel):
    arbre = tree.DecisionTreeClassifier()
    return arbre.fit(data, classLabel)




In [5]:


def main():
    data = Data(load_data("glass.data"))
    X, y = data.sliceLabel()
    arbre = decision_tree(X, y)
    tree.plot_tree(arbre)
    plt.show()
